In [4]:
import numpy as np

### Recurrent Neural Networks (RNNs)

RNNs funcionam como uma rede neural tradicional, tendo como diferença o uso de dois tipos de entrada: a entrada atual (X) e as entradas anteriormente processadas pela rede (h), tendo cada uma seu vetor de pesos e bias. Assim, para implmementar uma camada RNN, são necessários como etapa:
* Inicializar cada peso aleatoriamente
* Leitura e processamento da base
* Multiplicar pesos e entradas do vetor X
* Multiplicar pesos e entradas do vetor h
* Junção aditiva formada pelas multiplicações das etapas anteriores
* Função de ativação tanh para camadas escondidas
* Calcular saída da rede

![alt text](imgs/LSTM3-SimpleRNN.png "Title")

### Inicializar Pesos

In [11]:
hidden_size = 10 
seq_length = 5

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 
Whh = np.random.randn(hidden_size, hidden_size)*0.01 
Why = np.random.randn(vocab_size, hidden_size)*0.01 
bh = np.zeros((hidden_size, 1)) 
by = np.zeros((vocab_size, 1)) 

### Leitura e processamento do dataset

In [6]:
# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data)
print('data has %d characters, %d unique.' % (data_size, vocab_size))

Deep Learning Capacitacao LAVID - LSTM E RNN

data has 45 characters, 25 unique.


### RNN Forward Pass

In [7]:
def forward_rnn(X,h,Wxh,Whh,Why,bh,by):
    j_aditiva = np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x)
    h = np.tanh(j_aditiva + bh)
    y = softmax(np.dot(Why, h) + by)

### LSTM

Para implementar LSTM, a essência é a mesma, a rede terá uma entrada X atual e as entradas anteriores h, cada uma com seus respectivos pesos e bias. Entretanto, a rede divide-se em quatro camadas principais tidas como forget gate, input e update gate, output gate. Cada camada possui uma implementação diferenciada para filtrar os dados da rede e considerar o que é importante para o aprendizado. Assim, tem-se como etapas:
* Iniciar aleatoriamente os pesos e bias de cada camada
* Multiplicar cada entrada individualmente pelo peso Wf (peso da camada forget gate), somando-a em uma junção aditiva e passando pela função de ativação sigmoid e adicionando-se o bias dessa camada
* Multiplicar cada entrada individualmente pelo peso Wi (peso da camada input gate), somando-a em uma junção aditiva e passando pela função de ativação sigmoid e adicionando-se o bias dessa camada
* Multiplicar cada entrada individualmente pelo peso Wu (peso da camada update gate), somando-a em uma junção aditiva e passando pela função de ativação tanh e adicionando-se o bias dessa camada
* Atualizar a memória da célula multiplicando a memória anterior pela camada forget gate
* Atualizar a memória da célula somando-se os resultados obtidos pelas entradas input e update gate
* Multiplicar cada entrada individualmente pelo peso da camada de saida Wo, somando-a em uma junção aditiva e passando pela função de ativação sigmoid
* Atualizar a memória da célula com os valores de saída
* Calcular a saída y da célula

![alt text](imgs/LSTM3-chain.png "Title")

### Função de Ativação e predição

In [23]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [21]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

### Inicializar pesos

In [9]:
Wf = np.random.randn(hidden_size, hidden_size+vocab_size)
Wi = np.random.randn(hidden_size, hidden_size+vocab_size)
Wo = np.random.randn(hidden_size, hidden_size+vocab_size)
Wc = np.random.randn(hidden_size, hidden_size+vocab_size)
Wy = np.random.randn(vocab_size,hidden_size)
bf = np.random.randn(hidden_size,1)
bi = np.random.randn(hidden_size,1)
bo = np.random.randn(hidden_size,1)
bc = np.random.randn(hidden_size,1)
by = np.random.randn(vocab_size,1)

### Forward Pass LSTM

In [10]:
def forward_lstm(X,h,Wf,bf,Wi,bi,Wu,bu,c_prev):
    # Gates
    forget_gate = sigmoid(np.dot(Wf,X)+np.dot(Wf,h)+bf)
    input_gate  = sigmoid(np.dot(Wi,X)+np.dot(Wi,h)+bi)
    update_gate = np.tanh(np.dot(Wu,X)+np.dot(Wu,h)+bu)
    
    #Atualiza celula
    c_next = forget_gate*c_prev+update_gate*input_gate
    output = sigmoid(np.dot(Wo,X)+bo)
    cell = output*np.tanh(c_next)
    
    # Compute prediction of the LSTM cell (≈1 line)
    y = softmax(np.dot(Wy, cell) + by)

### Keras

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Embedding,  BatchNormalization
import numpy as np
from scipy.io import arff
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold

model = Sequential()
model.add(Embedding(vocab_size, vocab_size, input_length=sequence_length))
model.add(LSTM(10, return_sequences=True,
               input_shape=(sequence_length, len(data))))  
model.add(Dropout(0.9))               
model.add(LSTM(10, return_sequences=True))  
model.add(LSTM(10))  
model.add(Dropout(0.9))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)

classes = model.predict(x_test)

score, acc = model.evaluate(x_test, y_test, verbose = 1)